In [152]:
# imports
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [105]:
feedback = pd.read_csv('title_conference.csv',index_col=None)

feedback = shuffle(feedback)
feedback.reset_index(inplace = True)
feedback.drop('index', axis=1, inplace=True)

feedback.head(3)

Description     Class
0  Innovation in Database Management: Computer Sc...      VLDB
1  High performance prime field multiplication fo...     ISCAS
2  enchanted scissors: a scissor interface for su...  SIGGRAPH

In [110]:
# Label Encoder

le = preprocessing.LabelEncoder()
feedback['Class'] = le.fit_transform(feedback['Class'])

feedback.head(3)
# For inverse transform
#le.inverse_transform([0])

In [108]:
class_count = feedback['Class'].value_counts()
class_count

ISCAS       864
INFOCOM     515
VLDB        423
WWW         379
SIGGRAPH    326
Name: Class, dtype: int64

In [113]:
from keras.utils import to_categorical
target=feedback.Class.values
y=to_categorical(target)

In [139]:
max_features = 1300
max_words = 50
batch_size = 128
epochs = 3
num_classes=5

In [140]:
X_train , X_val , Y_train , Y_val = train_test_split(feedback['Description'],y,test_size = 0.25)

In [141]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,GRU,LSTM,Embedding
from keras.optimizers import Adam
from keras.layers import SpatialDropout1D,Dropout,Bidirectional,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [142]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)


X_train =pad_sequences(X_train, maxlen=max_words)
X_val = pad_sequences(X_val, maxlen=max_words)

In [143]:
#X_test = tokenizer.texts_to_sequences(test['Phrase'])
#X_test =pad_sequences(X_test, maxlen=max_words)

<a id='cnn'></a>
## <center>CNN</center>

In [144]:
model5_CNN= Sequential()
model5_CNN.add(Embedding(max_features,100,input_length=max_words))
model5_CNN.add(Dropout(0.2))
model5_CNN.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model5_CNN.add(GlobalMaxPooling1D())
model5_CNN.add(Dense(128,activation='relu'))
model5_CNN.add(Dropout(0.2))
model5_CNN.add(Dense(num_classes,activation='softmax'))
model5_CNN.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model5_CNN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           130000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 64)            19264     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

In [145]:
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)

history5=model5_CNN.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=100, batch_size=batch_size, verbose=1, callbacks = [early_stop])

Epoch 1/100
15/15 [==============================] - 1s 37ms/step - loss: 1.5887 - accuracy: 0.2693 - val_loss: 1.5268 - val_accuracy: 0.3413
Epoch 2/100
15/15 [==============================] - 0s 26ms/step - loss: 1.5086 - accuracy: 0.3462 - val_loss: 1.4784 - val_accuracy: 0.3413
Epoch 3/100
15/15 [==============================] - 0s 26ms/step - loss: 1.4441 - accuracy: 0.3428 - val_loss: 1.3714 - val_accuracy: 0.3445
Epoch 4/100
15/15 [==============================] - 0s 26ms/step - loss: 1.2909 - accuracy: 0.3936 - val_loss: 1.1710 - val_accuracy: 0.5534
Epoch 5/100
15/15 [==============================] - 0s 25ms/step - loss: 1.0293 - accuracy: 0.6534 - val_loss: 0.9669 - val_accuracy: 0.6140
Epoch 6/100
15/15 [==============================] - 0s 26ms/step - loss: 0.7629 - accuracy: 0.7663 - val_loss: 0.8581 - val_accuracy: 0.6842
Epoch 7/100
15/15 [==============================] - 0s 26ms/step - loss: 0.5676 - accuracy: 0.8240 - val_loss: 0.8138 - val_accuracy: 0.7113
Epoch 

In [146]:
y_pred5=model5_CNN.predict_classes(X_train)
#sub.Sentiment=y_pred5
#sub.to_csv('sub5_CNN.csv',index=False)
#sub.head()

In [147]:
y_pred5

array([4, 1, 1, ..., 3, 1, 1], dtype=int64)

In [148]:
test_txt = pd.Series('Innovation in Database Management: Computer Science vs. Engineering.') #test['Phrase'][0])

In [149]:
test_txt = tokenizer.texts_to_sequences(test_txt)
test_txt = pad_sequences(test_txt, maxlen=max_words)

In [150]:
model5_CNN.predict_classes(test_txt)

array([3], dtype=int64)

In [151]:
le.inverse_transform([3])

array(['VLDB'], dtype=object)